# SQL (SQLAlchemy) | SQL (SQLAlchemy)

> [Structured Query Language (SQL)](https://en.wikipedia.org/wiki/SQL)は、プログラミングで使用され、リレーショナルデータベース管理システム（RDBMS）で保持されるデータの管理や、リレーショナルデータストリーム管理システム（RDSMS）でのストリーム処理に特化して設計されたドメイン固有の言語です。特に、エンティティ間や変数間の関係を含む構造化データの取り扱いに役立ちます。
>
> > [Structured Query Language (SQL)](https://en.wikipedia.org/wiki/SQL) is a domain-specific language used in programming and designed for managing data held in a relational database management system (RDBMS), or for stream processing in a relational data stream management system (RDSMS). It is particularly useful in handling structured data, i.e., data incorporating relations among entities and variables.

> [SQLAlchemy](https://github.com/sqlalchemy/sqlalchemy)は、MITライセンスの下でリリースされたPythonプログラミング言語用のオープンソース`SQL`ツールキットおよびオブジェクトリレーショナルマッパー（ORM）です。
>
> > [SQLAlchemy](https://github.com/sqlalchemy/sqlalchemy) is an open-source `SQL` toolkit and object-relational mapper (ORM) for the Python programming language released under the MIT License.

このノートブックでは、`SQLAlchemy`によってサポートされる任意のデータベースにチャット履歴を保存できる`SQLChatMessageHistory`クラスについて説明しています。

> This notebook goes over a `SQLChatMessageHistory` class that allows to store chat history in any database supported by `SQLAlchemy`.

`SQLite`以外のデータベースを使用する場合は、対応するデータベースドライバーをインストールする必要があります。

> Please note that to use it with databases other than `SQLite`, you will need to install the corresponding database driver.




## Basic Usage | 基本的な使用方法

ストレージを使用するために必要なのは、たった2つのことだけです：

> To use the storage you need to provide only 2 things:

1. セッションID - セッションを一意に識別するための識別子で、ユーザー名、メールアドレス、チャットIDなどがこれに該当します。

   > Session Id - a unique identifier of the session, like user name, email, chat id etc.

2. 接続文字列 - データベース接続を指定する文字列です。これはSQLAlchemyのcreate\_engine関数に渡されます。

   > Connection string - a string that specifies the database connection. It will be passed to SQLAlchemy create\_engine function.

3. `SQLAlchemy` Pythonパッケージをインストールしてください。

   > Install `SQLAlchemy` python package.





In [ ]:
!pip install SQLAlchemy

In [1]:
from langchain.memory.chat_message_histories import SQLChatMessageHistory

chat_message_history = SQLChatMessageHistory(
    session_id="test_session", connection_string="sqlite:///sqlite.db"
)

chat_message_history.add_user_message("Hello")
chat_message_history.add_ai_message("Hi")

In [2]:
chat_message_history.messages

[HumanMessage(content='Hello', additional_kwargs={}, example=False),
 AIMessage(content='Hi', additional_kwargs={}, example=False)]

## Custom Storage Format | カスタムストレージフォーマット

デフォルトでは、セッションIDとメッセージ辞書のみがテーブルに保存されます。

> By default, only the session id and message dictionary are stored in the table.

しかし、時にはメッセージの日付、著者、言語などの追加情報を保存したいと思うこともあるでしょう。

> However, sometimes you might want to store some additional information, like message date, author, language etc.

それを行うために、**BaseMessageConverter** インターフェースを実装してカスタムメッセージコンバーターを作成することができます。

> To do that, you can create a custom message converter, by implementing **BaseMessageConverter** interface.




In [3]:
from datetime import datetime
from typing import Any

from langchain.memory.chat_message_histories.sql import BaseMessageConverter
from langchain.schema import AIMessage, BaseMessage, HumanMessage, SystemMessage
from sqlalchemy import Column, DateTime, Integer, Text
from sqlalchemy.orm import declarative_base

Base = declarative_base()


class CustomMessage(Base):
    __tablename__ = "custom_message_store"

    id = Column(Integer, primary_key=True)
    session_id = Column(Text)
    type = Column(Text)
    content = Column(Text)
    created_at = Column(DateTime)
    author_email = Column(Text)


class CustomMessageConverter(BaseMessageConverter):
    def __init__(self, author_email: str):
        self.author_email = author_email

    def from_sql_model(self, sql_message: Any) -> BaseMessage:
        if sql_message.type == "human":
            return HumanMessage(
                content=sql_message.content,
            )
        elif sql_message.type == "ai":
            return AIMessage(
                content=sql_message.content,
            )
        elif sql_message.type == "system":
            return SystemMessage(
                content=sql_message.content,
            )
        else:
            raise ValueError(f"Unknown message type: {sql_message.type}")

    def to_sql_model(self, message: BaseMessage, session_id: str) -> Any:
        now = datetime.now()
        return CustomMessage(
            session_id=session_id,
            type=message.type,
            content=message.content,
            created_at=now,
            author_email=self.author_email,
        )

    def get_sql_model_class(self) -> Any:
        return CustomMessage


chat_message_history = SQLChatMessageHistory(
    session_id="test_session",
    connection_string="sqlite:///sqlite.db",
    custom_message_converter=CustomMessageConverter(author_email="test@example.com"),
)

chat_message_history.add_user_message("Hello")
chat_message_history.add_ai_message("Hi")

In [4]:
chat_message_history.messages

[HumanMessage(content='Hello', additional_kwargs={}, example=False),
 AIMessage(content='Hi', additional_kwargs={}, example=False)]

また、session\_id列の名前を変更することもできます。その場合は、`session_id_field_name`パラメータを指定する必要があります。

> You also might want to change the name of session\_id column. In this case you'll need to specify `session_id_field_name` parameter.


